In [ ]:
'''
This notebook is a companion to the Backpropagation from Scratch project and is designed to strengthen your intuition behind computing gradients by hand. It includes small, focused
exercises where you:

📐 Compute derivatives manually using calculus

🔍 Verify results using PyTorch’s autograd engine

📌 What You'll Learn
    -The chain rule applied in practice
    -Gradient flow through arithmetic operations
    -Manual gradient computation of:
        -> Scalars
        -> Vectors
        -> Activation functions (e.g. sigmoid, tanh)
    -How to compare with PyTorch’s .backward() to validate your results


⚙️ Requirements
    -Python 3.x
    -PyTorch
    -Jupyter Notebook

Install using pip:
pip install torch notebook

🚀 How to Run
git clone https://github.com/nimxrah23/backprop-from-scratch.git
cd backprop-from-scratch
jupyter notebook
Open the file derivatives_practice.ipynb to start solving the exercises.

🎯 Who Is This For?
This notebook is perfect for:

ML/DL beginners wanting to build intuition
Students trying to understand how autograd works under the hood
Anyone who wants to visualize and verify each gradient computation



In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  yours = []
  yours.append(-3*a**2 - 0.5*a**-0.5)
  yours.append(cos(3*b)*3 + 2.5*b**1.5)
  yours.append(-1.0*-1.0*c**-2)
  return yours # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell
def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5
# -----------
a= 2
b = 3
c = 4
h = 0.000001
numerical_grad = []
ans0 = f(a, b, c)
ans1 = f(a+h, b, c)
ans2 = f(a, b+h, c)
ans3 = f(a, b, c+h)
numerical_grad.append((ans1-ans0)/h)
numerical_grad.append((ans2-ans0)/h)
numerical_grad.append((ans3-ans0)/h)


# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

# -----------
h = 0.000001
a = 2
b = 3
c = 4
numerical_grad2 = [] # TODO
ans0 = f(a, b, c)
ans1 = (f(a+h, b, c) - f(a-h, b ,c))/(2*h)
numerical_grad2.append(ans1)
ans2 = (f(a, b+h, c) - f(a, b-h ,c))/(2*h)
numerical_grad2.append(ans2)
ans3 = (f(a, b, c+h) - f(a, b ,c-h))/(2*h)
numerical_grad2.append(ans3)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log
import math
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
      self.grad += (other * (self.data ** (other-1))) * out.grad
    out._backward = _backward
    return out

  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x)-1) / (math.exp(2*x)+1)
    out = Value(t, (self, ), 'tanh')

    def _backward():
      self.grad += (1-t**2)*out.grad

    out._backward = _backward
    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad

    out._backward = _backward
    return out

  def log(self):
    x = self.data
    out = Value(math.log(x), (self, ), 'log')

    def _backward():
      self.grad += (1/x)*out.grad

    out._backward = _backward
    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:

# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad = True)
exps = torch.exp(logits)
probs = exps/exps.sum()
loss = -torch.log(probs[3])
loss.backward()

print("Loss:", loss.item())
print("Gradients:")
for i in range(len(logits)):
    print(f"logits[{i}].grad = {logits.grad[i].item()}")



Loss: 2.1755154132843018
Gradients:
logits[0].grad = 0.041772566735744476
logits[1].grad = 0.8390244245529175
logits[2].grad = 0.005653302185237408
logits[3].grad = -0.8864503502845764
